In [28]:
import numpy as np
from pywarpx import picmi
from pywarpx.picmi import Simulation
from pydantic import BaseModel, ConfigDict

import json

constants = picmi.constants

In [29]:
def cosd(x):
    return np.cos(np.deg2rad(x))


def sind(x):
    return np.sin(np.deg2rad(x))

In [30]:
class CustomSimulation(BaseModel):

    dim: int = None
    diag: bool = True

    # Plasma parameters
    n0: float = None
    """plasma density (m^-3)"""

    # Plasma species parameters
    m_ion_norm: float = 400
    """Ion mass (electron masses)"""
    m_ion: float = None
    """Ion mass (kg)"""
    v_ti: float = None
    """Ion thermal velocity (m/s)"""

    # Spatial domain
    nz: int = None  #: number of cells in z direction
    nx: int = None  #: number of cells in x (and y) direction

    # Numerical parameters
    nppc: int = 64
    """Seed number of particles per cell"""
    diag_steps: int = None
    """The simulation step interval at which to output diagnostic"""

    _sim: Simulation = None

    model_config = ConfigDict(
        extra="allow",
    )

    def setup_particle(self):
        """setup the particle"""
        return self

    def setup_grid(self):
        """Setup geometry and boundary conditions"""
        if self.dim == 1:
            grid_object = picmi.Cartesian1DGrid
        elif self.dim == 2:
            grid_object = picmi.Cartesian2DGrid
        else:
            grid_object = picmi.Cartesian3DGrid

        number_of_cells = [self.nx, self.nx, self.nz][-self.dim :]
        boundary_conditions = ["periodic"] * self.dim

        self._grid = grid_object(
            number_of_cells=number_of_cells,
            lower_bound=[-self.Lx / 2.0, -self.Lx / 2.0, 0][-self.dim :],
            upper_bound=[self.Lx / 2.0, self.Lx / 2.0, self.Lz][-self.dim :],
            lower_boundary_conditions=boundary_conditions,
            upper_boundary_conditions=boundary_conditions,
        )
        return self

    def setup_field(self):
        pass

    def setup_field_solver(self):
        pass

    def setup_diag(self):
        ...

    def setup_run(self):
        """Setup simulation components."""
        self.setup_grid().setup_field_solver().setup_field()
        self.setup_particle()
        if self.diag:
            self.setup_diag() 
        self._sim.initialize_inputs()

    def dump(self, file="sim_parameters.json"):
        d = dict(self.model_dump())
        with open(file, "w") as f:
            json.dump(d, f)

In [31]:
class HybridSimulation(CustomSimulation):

    beta: float = 0.1
    """Plasma beta"""  # used to calculate temperature

    vA: float = None
    """Alfven speed"""

    plasma_resistivity: float = 1e-7  # TODO: find a good value
    """Plasma resistivity"""

    T_plasma: float = None
    Te: float = None
    """Electron temperature in (eV)"""

    t_ci: float = None
    """Ion cyclotron period (s)"""
    d_i: float = None
    """Ion inertial length (m)"""

    # Numerical parameters
    time_norm: float = 100.0
    """Simulation temporal length (ion cyclotron periods)"""
    dt_norm: float = 1 / 64
    """Time step (ion cyclotron periods)"""
    diag_time_norm: float = 1
    """Time interval at which to output diagnostic (ion cyclotron periods)"""

    Lz_norm: float = None
    Lx_norm: float = None
    """Spatial domain length (ion skin depths)"""
    dz_norm: float = 1 / 8
    """Cell size (ion skin depths)"""

    def post_init(self):
        """This function is called after the object is initialized"""
        self._sim.time_step_size = self.dt_norm * self.t_ci

        self.dz = self.dz_norm * self.d_i        
        self.nz = int(self.Lz_norm / self.dz_norm)
        self.nx  = int(self.Lx_norm / self.dz_norm)
        
        self.Lz = self.Lz_norm * self.d_i
        self.Lx = self.Lx_norm * self.d_i
        

    def setup_run(self):
        self._sim.current_deposition_algo = "direct"
        self._sim.particle_shape = 1
        super().setup_run()

    def setup_field_solver(self):
        """Setup field solver"""

        self._sim.solver = picmi.HybridPICSolver(
            grid=self._grid,
            Te=self.Te,
            n0=self.n0,
            plasma_resistivity=self.plasma_resistivity,
            n_floor = 0.01 * self.n0,
        )
        return self

    def setup_diag(self):
        self.diag_steps = int(self.diag_time_norm / self.dt_norm)

        field_diag = picmi.FieldDiagnostic(
            grid=self._grid,
            period=self.diag_steps,
        )

        self._sim.add_diagnostic(field_diag)

        line_diag = picmi.ReducedDiagnostic(
            diag_type="FieldProbe",
            probe_geometry="Line",
            x_probe=-self.Lx / 2,
            x1_probe=self.Lx / 2,
            z_probe=self.Lz / 2,
            z1_probe=self.Lz / 2,
            resolution=self.nx - 1,
            period=self.diag_steps,
        )
        # self._sim.add_diagnostic(line_diag)

In [32]:
def log_sim_info(sim: Simulation):
    """print out plasma parameters and numerical parameters."""
    print(
        f"Numerical parameters:\n"
        f"\tdt = {sim.time_step_size:.1e} s\n"
        f"\ttotal steps = {sim.max_steps:d}\n"
    )


def log_info(sim: HybridSimulation):
    """print out plasma parameters and numerical parameters."""
    log_sim_info(sim._sim)
    print(
        f"Initializing simulation with input parameters:\n"
        f"\tTe = {sim.Te:.3f} eV\n"
        f"\tn = {sim.n0/1e6:.1e} cm^-3\n"
        f"\tB0 = {sim.B0/1e-9:.2f} nT\n"
        f"\tM/m = {sim.m_ion_norm:.0f}\n"
    )
    print(
        f"Plasma parameters:\n"
        f"\td_i = {sim.d_i:.1e} m\n"
        f"\tt_ci = {sim.t_ci:.1e} s\n"
        f"\tv_ti = {sim.v_ti:.1e} m/s\n"
        f"\tvA = {sim.vA:.1e} m/s\n"
        f"\tvA/c = {sim.vA/constants.c}\n"
    )

Support `gaussian_parse_momentum_function`?


In [33]:
def init_field(
    k,
    B0,
    A,  #: relative amplitude
    theta=60,
):
    """
    Generate field with a wave propagating along the x axis at a large angle `theta` with respect to the background magnetic field lying in the x-z plane.

    The initial waveis an Alfven mode in which the magnetic field fluctuation points along the y and z axis and has a relative amplitude $A = \delta B_y / B_0$
    """

    B0x = B0 * cosd(theta)
    B0z = B0 * sind(theta)

    # dB0z = B0 * A * np.cos(k * 0)

    Bx_expression = f"{B0x}"
    By_expression = f"{A} * {B0} * cos({k} * x)"
    Bz_expression = f"{B0z}"

    return picmi.AnalyticInitialField(
        Bx_expression=Bx_expression,
        By_expression=By_expression,
        Bz_expression=Bz_expression,
    )


def init_plasma(
    vA,
    n0,
    v_ti,  #: ion thermal velocity
    k,
    B0,
    A,  #: relative amplitude
    theta=60,
):
    """
    The ion bulk and transverse velocity V remains parallel to the total transverse magnetic field.
    """
    B0x = B0 * cosd(theta)
    B0z = B0 * sind(theta)

    px_expression = vA * B0x / B0
    py_expression = f"{vA * A} * cos({k} * x)"
    pz_expression = vA * B0z / B0
    
    px_expression = 0 
    pz_expression = 0

    momentum_expressions = [px_expression, py_expression, pz_expression]

    return picmi.AnalyticDistribution(
        density_expression=n0,
        momentum_expressions=momentum_expressions,
        # rms_velocity=[v_ti] * 3,
    )

In [34]:
class AlfvenModes(HybridSimulation):
    test: bool = True
    # Applied field parameters
    dim: int = 2
    B0: float = 1e-3
    """Initial magnetic field strength (T)"""
    # n0: float = 100 * 1e6
    n0: float = None
    """Initial plasma density (m^-3)"""
    
    vA_over_c : float = 1e-3
    """ratio of Alfven speed and the speed of light"""

    A: float = 0.5  # relative amplitude

    # Spatial domain
    Lz_norm: float = 4
    Lx_norm: float = 128 # spatial domain length in x direction (ion skin depths)

    def model_post_init(self, __context):
        """Get input parameters for the specific case desired."""
        self._sim = Simulation(
            warpx_serialize_initial_conditions=True,
        )
        
        if self.test:
            self.nppc = 16
            self.m_ion_norm = 100

        # calculate various plasma parameters based on the simulation input
        self.get_plasma_quantities()

        self.post_init()
        self._sim.max_steps = int(self.time_norm / self.dt_norm)
        self.k = 2 * 2 * np.pi / self.Lx
        self.setup_run()
        log_info(self)
        self.dump()

    def setup_field(self):
        """Setup external field"""

        B_ext = init_field(k=self.k, B0=self.B0, A=self.A)

        self._sim.add_applied_field(B_ext)
        return self

    def setup_particle(self):
        """setup the particle"""

        dist = init_plasma(
            vA=self.vA, n0=self.n0, v_ti=self.v_ti, k=self.k, B0=self.B0, A=self.A
        )

        ions = picmi.Species(
            name="ions",
            charge_state=1,
            mass=self.m_ion,
            initial_distribution=dist,
        )

        self._sim.add_species(
            ions,
            layout=picmi.PseudoRandomLayout(
                grid=self._grid, n_macroparticles_per_cell=self.nppc
            ),
        )
        return self

    def get_plasma_quantities(self):
        """Calculate various plasma parameters based on the simulation input."""
        # Ion mass (kg)
        self.m_ion = self.m_ion_norm * constants.m_e

        # Cyclotron angular frequency (rad/s) and period (s)
        self.w_ci = constants.q_e * abs(self.B0) / self.m_ion
        self.t_ci = 2.0 * np.pi / self.w_ci

        # Alfven speed (m/s): vA = B / sqrt(mu0 * n * (M + m)) = c * omega_ci / w_pi        
        # self.vA = self.B0 / np.sqrt(
            # constants.mu0 * self.n0 * (self.m_ion + constants.m_e)
        # )
        
        self.vA = self.vA_over_c * constants.c
        
        self.n0 = (
            (self.B0 / self.vA)**2 / (constants.mu0 * (self.m_ion + constants.m_e))
        )

        # Ion plasma frequency (rad/s)
        self.w_pi = np.sqrt(constants.q_e**2 * self.n0 / (self.m_ion * constants.ep0))

        # Skin depth (m): inertial length
        self.d_i = constants.c / self.w_pi

        # Ion thermal velocity (m/s) from beta = 2 * (v_ti / vA)**2
        self.v_ti = np.sqrt(self.beta / 2.0) * self.vA

        # Temperature (eV) from thermal speed: v_ti = sqrt(kT / M)
        self.T_plasma = self.v_ti**2 * self.m_ion / constants.q_e  # eV
        self.Te = self.T_plasma

        # Larmor radius (m)
        self.rho_i = self.v_ti / self.w_ci

In [35]:
simulation = AlfvenModes()

Numerical parameters:
	dt = 1.4e-08 s
	total steps = 2560

Initializing simulation with input parameters:
	Te = 2.555 eV
	n = 9.6e+10 cm^-3
	B0 = 1000000.00 nT
	M/m = 100

Plasma parameters:
	d_i = 1.7e-01 m
	t_ci = 3.6e-06 s
	v_ti = 6.7e+04 m/s
	vA = 3.0e+05 m/s
	vA/c = 0.001



In [48]:
simulation.diag_steps

print(f"""
    {simulation._sim.max_steps}
    {simulation._sim.time_step_size}
    """
)


    2560
    1.3954635753430507e-08
    


In [ ]:
simulation._sim.write_input_file()
# simulation.initialize_warpx()

In [ ]:
simulation._sim.step()

## Test


In [ ]:
from plasmapy.formulary import plasma_frequency, inertial_length
import astropy.units as u

In [ ]:
def test():
    plasma_frequency(simulation.n0 * u.m**-3, "p+")